# Small

In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import h5py
from os import path
import collections as col
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = '{:.4f}'.format

In [2]:
small_data = h5py.File('small_data.hdf5')

In [3]:
list(small_data)

[]

In [4]:
small = analyze.Study('small', number_of_replicates=10, data_file=small_data)

In [5]:
run_id = 'small'
generations_of_random_mating = 10
number_of_qtl = 30
number_of_replicates = 10
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 1000
mating_pop_size = len(founders)*os_per_pair
recombination_rates = [0.01]*1478
sample_size = 1000

In [6]:
prefounders = sim.loadPopulation('bia_prefounders.pop')

In [7]:
prefounders.infoFields()

('ind_id',
 'father_id',
 'mother_id',
 'fitness',
 'p',
 'g',
 'generation',
 'replicate')

In [8]:
sim.tagID(prefounders, reset=True)

In [9]:
prefounders.popSize()

26

In [10]:
multi_prefounders = sim.Simulator(prefounders, 10, stealPops=False)

In [11]:
magic = breed.MAGIC(multi_prefounders, founders, recombination_rates)

In [12]:
magic.generate_f_one(founders, os_per_pair)

In [13]:
mrc = breed.MultiRandomCross(multi_prefounders, 4, 1000)

In [14]:
mother_choices, father_choices = mrc.determine_random_cross()

In [15]:
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    mother_choices, father_choices)

In [16]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[mating_pop_size],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [17]:
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 2000)

In [18]:
final_mothers, final_fathers = final_mrc.determine_random_cross()

In [19]:
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    final_mothers, final_fathers)

In [20]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[mating_pop_size],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

# Random Mating Phase

In [21]:
multi_prefounders.evolve(
    matingScheme=sim.RandomMating(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
        subPopSize=[mating_pop_size]),
    gen=100,
)

(100, 100, 100, 100, 100, 100, 100, 100, 100, 100)

In [22]:
sample_library = small.collect_samples(multi_prefounders, [sample_size])

In [23]:
for rep_id, sample_list in sample_library.items():
    sim.stat(sample_list[0], numOfSegSites=sim.ALL_AVAIL, vars=['numOfSegSites', 'segSites'])
    sim.stat(sample_list[0], alleleFreq=sim.ALL_AVAIL)

In [24]:
list_of_segs = [sample_library[i][0].dvars().segSites for i in range(number_of_replicates)]

In [25]:
commonly_segregating_loci = list(set(sample_library[0][0].dvars().segSites).intersection(*list_of_segs))

In [26]:
len(commonly_segregating_loci)

942

In [27]:
sample = sample_library[0][0]

In [28]:
astates = small.gather_allele_data(sample)

In [29]:
alleles = np.array([astates[:, 1], astates[:, 2]]).T

In [30]:
segregating_loci = np.array(commonly_segregating_loci)

In [31]:
trait = parameters.Trait()

In [32]:
qtl = sorted(list(random.sample(list(segregating_loci), number_of_qtl)))

In [33]:
allele_effects = trait.construct_allele_effects_table(alleles, qtl, random.expovariate, 1)

In [34]:
allele_effects[qtl]

array([[   63.   ,     0.   ,     0.031,     2.   ,     0.399],
       [   99.   ,     0.   ,     0.104,     1.   ,     0.349],
       [  124.   ,     1.   ,     0.543,     3.   ,     0.133],
       [  264.   ,     1.   ,     0.764,     3.   ,     0.899],
       [  270.   ,     0.   ,     1.245,     2.   ,     0.37 ],
       [  280.   ,     0.   ,     2.275,     2.   ,     1.347],
       [  313.   ,     0.   ,     0.902,     2.   ,     2.036],
       [  333.   ,     1.   ,     1.341,     3.   ,     0.543],
       [  336.   ,     2.   ,     1.043,     3.   ,     7.344],
       [  413.   ,     0.   ,     2.116,     3.   ,     1.147],
       [  475.   ,     0.   ,     0.02 ,     3.   ,     1.074],
       [  480.   ,     0.   ,     0.448,     2.   ,     0.117],
       [  491.   ,     0.   ,     0.308,     2.   ,     0.17 ],
       [  517.   ,     1.   ,     1.58 ,     3.   ,     0.978],
       [  585.   ,     1.   ,     0.261,     3.   ,     0.978],
       [  616.   ,     1.   ,     0.417,

In [35]:
ae_array = trait.construct_ae_array(allele_effects, qtl)

# Storing Data

In [ ]:
del small_data['allele/states']
del small_data['segregating_loci']
del small_data['qtl']
del small_data['allele/effects']
del small_data['recombination_rates']
del small_data['allele/effects_array']

In [36]:
small_data['allele/states'] = astates
small_data['segregating_loci'] = np.array(commonly_segregating_loci)
small_data['qtl'] = np.array(qtl)
small_data['allele/effects'] = allele_effects
small_data['recombination_rates'] = np.array(recombination_rates)
small_data['allele/effects_array'] = ae_array

In [ ]:
del small_data['allele/frequency/replicate']
del small_data['trait/g/replicate']
del small_data['trait/p/replicate']

In [37]:
for rep, sample_list in sample_library.items():
    small_data['allele/frequency/replicate/' + str(rep)] = small.gather_allele_frequencies(sample_list[0], astates)
    operators.calculate_g(sample_list[0], ae_array)
    operators.calculate_error_variance(sample_list[0], 0.7)
    operators.calculate_p(sample_list[0])
    small_data['trait/g/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'), 
                                                            sample_list[0].indInfo('g')]).T
    small_data['trait/p/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'),
                                                          sample_list[0].indInfo('p')]).T
    

In [38]:
small_data['trait'].attrs['heritability'] = np.array([0.7])

In [39]:
minor_alleles = np.array(small_data['allele/states'])[:, 3]

In [40]:
indir = '/home/vakanas/tassel-5-standalone/input'
outdir = '/home/vakanas/tassel-5-standalone/output'

In [41]:
print("Computing TASSEL Input:")
for rep, sample_list in sample_library.items():
    name = small.run_id + '_' + str(rep)
    print("{current_rep}\t".format(current_rep=str(rep)))
    minor_allele_fs = np.array(small_data['allele/frequency/replicate/' + str(rep)])[segregating_loci, 3]
    gwas = analyze.GWAS(sample_list[0], segregating_loci, minor_alleles, 'small')
    cm = gwas.calculate_count_matrix(count_matrix_file_name=path.join(indir, name+'_count_matrix.txt'))
    ps, svd = gwas.pop_struct_eigendecomp(cm)
    gwas.population_structure_formatter(ps, svd, number_of_pcs=2, 
                                        pop_struct_file_name=path.join(indir, name+'_structure_matrix.txt'))
    gwas.trait_formatter(trait_file_name=path.join(indir, name+'_phenotype_vector.txt'))
    gwas.calc_kinship_matrix(cm, minor_allele_fs, kinship_matrix_file_name=path.join(indir, name+'_kinship_matrix.txt'))
    gwas.hapmap_formatter(hapmap_file_name=path.join(indir, name+'_simulated_hapmap.txt'))
    gwas.single_gen_multi_rep_tassel_config(rep, 'gwas_pipeline.xml', output_prefix=name+'_out_')

Computing TASSEL Input:
0	
1	
2	
3	
4	
5	
6	
7	
8	
9	


# Run TASSEL at This Point

Contents of bash script to automate TASSEL via configuration files.

### simulated_mlm.sh

```bash
#!/bin/bash


echo "Run ID: $1, Number of Replicates $2"
run_id=$1
number_of_replicates=$2
final_rep_index="$((number_of_replicates - 1))"

echo "Beginning TASSEL analysis of Run ID: $run_id"
echo "Number of Replicates: $number_of_replicates"
echo "First configuration file: small_0_gwas_pipeline.xml"onca
echo "Final configuration file: small_"$final_rep_index"_gwas_pipeline.xml"

for i in `seq 0 $final_rep_index`
do
    config_file_name=$run_id$i"_gwas_pipeline.xml"
    echo "$config_file_name"
    ./run_pipeline.pl -Xmx6g -configFile $config_file_name
done


```

### Example output: 
+ small_0_out_1.txt
+ small_0_out_2.txt
+ small_0_out_3.txt
+ small_1_out_1.txt
+ ...
+ small_9_out_3.txt

# Use R Qvalue package to get Qvalues

Contents of R script to obtain qvalues for p column of TASSEL results

```R
#!/usr/bin/env Rscript

library(qvalue)
library(ggplot2)
library(gap)

args = commandArgs(trailingOnly=TRUE)

# test to determine if the file name parameter is supplied to the script
if (length(args)==0) {
  stop("At least one argument must be suppled (input file).\n", call.=FALSE)
}
#setwd("/home/vakanas/tassel-5-standalone/output")  

run_id = args[1]
file_name_match_pattern = paste(run_id, "(.*)_2.txt", sep='')
file_names = list.files(pattern = file_name_match_pattern)

for(n in file_names) {
    print(n)
    input_file_name = n
    run_id_prefix_terminus = nchar(input_file_name) - 5
    run_id_prefix = substring(input_file_name, 1, run_id_prefix_terminus)
    output_file_name = paste(run_id_prefix, 'q_values.txt', sep='')
    print(output_file_name)
    results_header = scan(input_file_name, what="character", nlines=1, sep="\t")
    gwas_results = read.table(input_file_name, header=F, row.names = NULL, skip=2)
    colnames(gwas_results) = results_header
    pvalues = gwas_results$p
    qobj = qvalue(p = pvalues)
    qvalues = data.frame(qobj$qvalues)
    colnames(qvalues) = "q"
    rownames(qvalues) = gwas_results$Marker
    write.table(qvalues, output_file_name, sep="\t")
}

```

# Analysis of TASSEL Results: Comutation of Power & FPR

## Subsetting Raw TASSEL Results and Data Storage
    Each replicate has an associated set of TASSEL output files. The raw
    results are modified and stored in the run's HDF5 file

## Statistical Power and False Positive Rate

In [ ]:
power_fprs = small.calculate_power_false_positive_rate(qtl, allele_effects, 10, 
                                                       hdf5_file=small_data)

In [ ]:
power_fprs

In [ ]:
res = np.array((small_data['tassel/test/replicate/0']))

In [ ]:
np.where(res[:, -1] < 0.05)

In [ ]:
allele_effects[908]

In [ ]:
qtl

## Allele Effects: Estimated vs. Actual
    Purpose of this table to compare TASSEL's effect size estimates
    with the known true values we assigned.

In [ ]:
gt = small.genotypic_effects_table(sample_library, ae_array,
                          segregating_loci, qtl, hdf5_file=small_data)

In [ ]:
expo_power_fpr_raw_data = analyze.collect_power_analysis_data(run_id, sample_sizes, number_of_replicates, concordant_segregating_loci, 'exponential')

In [ ]:
expo_power_fpr_raw_data[250]

In [ ]:
write_super_tables(expo_power_fpr_raw_data,
                  sample_sizes,
                  number_of_replicates,
                  'bacchus',
                  sub_run_id='exponential')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = study.calculate_power_fpr(expo_power_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
expo_results

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('bacchus_exponential_mean_and_stdev_power_fpr.csv', sep='\t')

In [ ]:
geo_results

In [ ]:
geometric_allele_effects_table

In [ ]:
exponential_allele_effects_table

In [ ]:
expo_results.to_csv("bacchus_exponential_power_fpr_results.txt", sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([geo_results.mean(), geo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_geometric_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = full_icecrown.calculate_power_fpr(expo_power_fpr_raw_data,
                                                                                      sample_sizes,
                                                                                      number_of_replicates,
                                                                                      number_of_qtl)

In [ ]:
expo_results

In [ ]:
expo_results.to_csv('full_icecrown_exponential_power_fpr_results.txt', sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_exponential_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
write_super_tables(expo_power_fpr_raw_data, sample_sizes, number_of_replicates, run_id, 'exponential')

In [ ]:
geo_aggregate_estimated_actual = pd.DataFrame([np.array(geo_agg_estimated), np.array(geo_agg_actual)], index=['estimated', 'actual']).T

In [ ]:
geo_aggregate_estimated_actual['estimated'] = geo_aggregate_estimated_actual['estimated'].apply(np.fabs)

In [ ]:
geo_aggregate_estimated_actual

In [ ]:
geo_corr = geo_aggregate_estimated_actual['estimated'].corr(geo_aggregate_estimated_actual['actual'])

In [ ]:
geo_agg_estimated

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_corr

In [ ]:
pwd

In [ ]:
geo_aggregate_estimated_actual.to_csv('full_icecrown_geometric_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
agg_estimated = []
agg_actual = []

In [ ]:
for rep in reps:
    for size in sample_sizes:
        sutable = sutable_collection[rep][size]
        droppable = list(sutable.ix[sutable.ix[:, 'difference'] == 0.0].index)
        qtloci = sutable.drop(droppable, axis=0)
        agg_estimated.extend(list(qtloci['add_effect']))
        agg_actual.extend(list(qtloci['difference']))

In [ ]:
aggregate_estimated_actual = pd.DataFrame([np.array(agg_estimated), np.array(agg_actual)], index=['estimated', 'actual']).T

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
aggregate_estimated_actual

In [ ]:
correlation_actual_vs_effects = aggregate_estimated_actual['estimated'].corr(aggregate_estimated_actual['actual'])

In [ ]:
aggregate_estimated_actual.to_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
expo_estimated_actual = pd.read_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t', index_col=0)

In [ ]:
expo_estimated_actual

In [ ]:
aggregate_estimated_actual

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_x = aggregate_estimated_actual['estimated']
geo_y = aggregate_estimated_actual['actual']

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(-0.2, 4))

In [ ]:
str_mat = pd.read_csv('/home/vakanas/tassel-5-standalone/input/small_0_structure_matrix.txt', 
                      sep='\t', index_col=0, skiprows=[1])

In [ ]:
str_vecs = np.array(str_mat)

In [ ]:
del p

In [ ]:
p = figure(title="Population Structure of Randomly Mated Population",)

In [ ]:
p.scatter(str_vecs[:, 0], str_vecs[:, 1])

In [ ]:
show(p)

In [ ]:
str_vecs

In [ ]:
g = np.array(small_data['trait/g/replicate/0'])[:20, 1]

In [ ]:
s = figure(title='PLot of G R 0')

In [ ]:
s.circle(list(range(20)), g)

In [ ]:
show(s)

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4))

x = np.array(expo_estimated_actual['estimated'])
y = np.array(expo_estimated_actual['actual'])

expo_plot.xaxis.axis_label = "Estimated"
expo_plot.yaxis.axis_label = "Actual"

In [ ]:
expo_plot.scatter(x, y)

In [ ]:
show(expo_plot)

In [ ]:
from bokeh.io import hplot

In [ ]:
geo_plot = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(0, 4), y_range=(0, 4))

In [ ]:
geo_x = aggregate_estimated_actual['actual']
geo_y = aggregate_estimated_actual['estimated']

In [ ]:
geo_plot.xaxis.axis_label = "Actual"
geo_plot.yaxis.axis_label = "Estimated"
geo_plot.scatter(geo_x, geo_y, x="Actual", y="Estimated")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4), y_range=(0, 4))

expo_x = np.array(expo_estimated_actual['actual'])
expo_y = np.array(expo_estimated_actual['estimated'])

expo_plot.xaxis.axis_label = "Actual"
expo_plot.yaxis.axis_label = "Estimated"
expo_plot.scatter(expo_x, expo_y)

In [ ]:
multi_plot = hplot(geo_plot, expo_plot)
show(multi_plot)

In [ ]:
output_file("multi_plot.png")

In [ ]:
ls